In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import json
from pathlib import Path
from pprint import pprint

# JSON 파일 로드
file_path = "data/people.json"
data = json.loads(Path(file_path).read_text(encoding="utf-8"))

# 데이터 확인
print("Original Data:")
pprint(data)

Original Data:
[{'address': {'city': '서울', 'street': '312번지', 'zipCode': '83795'},
  'age': 31,
  'carOwnership': True,
  'hobbies': ['요리', '음악 감상', '사진 촬영'],
  'isMarried': True,
  'name': '박시우',
  'phoneNumbers': ['483-4639-1933', '947-4179-7976']},
 {'address': {'city': '서울', 'street': '877번지', 'zipCode': '36780'},
  'age': 31,
  'carOwnership': True,
  'hobbies': ['여행', '음악 감상', '등산'],
  'isMarried': False,
  'name': '정수아',
  'phoneNumbers': ['337-5721-3227', '387-3768-9586']},
 {'address': {'city': '서울', 'street': '175번지', 'zipCode': '89067'},
  'age': 43,
  'carOwnership': True,
  'hobbies': ['등산', '독서', '게임'],
  'isMarried': False,
  'name': '최도윤',
  'phoneNumbers': ['354-5563-4638', '471-9212-1826']},
 {'address': {'city': '서울', 'street': '690번지', 'zipCode': '70635'},
  'age': 22,
  'carOwnership': False,
  'hobbies': ['여행', '등산', '게임'],
  'isMarried': False,
  'name': '정민준',
  'phoneNumbers': ['468-2796-2152', '922-5760-7030']},
 {'address': {'city': '서울', 'street': '151번지', '

  ### 청크 크기를 매우 작게 설정합니다. 예시를 위한 설정입니다.
    chunk_size=250, # 청크 간의 중복되는 문자 수를 설정합니다.
    chunk_overlap=50, # 문자열 길이를 계산하는 함수를 지정합니다.
    length_function=len, # 구분자로 정규식을 사용할지 여부를 설정합니다.
    is_separator_regex=False,

In [24]:
# JSON 데이터를 Document 형식으로 변환
documents = [
    Document(page_content=json.dumps(item, ensure_ascii=False), metadata={}) for item in data
]

# RecursiveCharacterTextSplitter 설정
text_splitter = RecursiveCharacterTextSplitter(
     # 청크 크기를 매우 작게 설정합니다. 예시를 위한 설정입니다.
    chunk_size=250,
    # 청크 간의 중복되는 문자 수를 설정합니다.
    chunk_overlap=50,
    # 문자열 길이를 계산하는 함수를 지정합니다.
    length_function=len,
    # 구분자로 정규식을 사용할지 여부를 설정합니다.
    is_separator_regex=False,
)

# 문서 분할
split_documents = text_splitter.split_documents(documents)

# 결과 출력
print("\nSplit Documents:")
for doc in split_documents:
    pprint({
        "content": doc.page_content,
        "metadata": doc.metadata
    })



Split Documents:
{'content': '{"name": "박시우", "age": 31, "isMarried": true, "carOwnership": '
            'true, "address": {"street": "312번지", "city": "서울", "zipCode": '
            '"83795"}, "phoneNumbers": ["483-4639-1933", "947-4179-7976"], '
            '"hobbies": ["요리", "음악 감상", "사진 촬영"]}',
 'metadata': {}}
{'content': '{"name": "정수아", "age": 31, "isMarried": false, "carOwnership": '
            'true, "address": {"street": "877번지", "city": "서울", "zipCode": '
            '"36780"}, "phoneNumbers": ["337-5721-3227", "387-3768-9586"], '
            '"hobbies": ["여행", "음악 감상", "등산"]}',
 'metadata': {}}
{'content': '{"name": "최도윤", "age": 43, "isMarried": false, "carOwnership": '
            'true, "address": {"street": "175번지", "city": "서울", "zipCode": '
            '"89067"}, "phoneNumbers": ["354-5563-4638", "471-9212-1826"], '
            '"hobbies": ["등산", "독서", "게임"]}',
 'metadata': {}}
{'content': '{"name": "정민준", "age": 22, "isMarried": false, "carOwnership": '
           

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json
from pathlib import Path

def load_and_split_people_data(file_path):
    # JSON 파일 로드
    try:
        data = json.loads(Path(file_path).read_text(encoding="utf-8"))
        print(f"데이터 로드 완료: {len(data)}개의 프로필")
    except Exception as e:
        print(f"파일 로드 중 에러 발생: {str(e)}")
        return None

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=250,
        chunk_overlap=50,
        length_function=len,
        is_separator_regex=False
    )
    
    chunked_data = []
    
    for person in data:
        try:
            # 기본 정보 추출
            basic_info = {
                "name": person.get("name"),
                "age": person.get("age")
            }
            
            # 상세 정보 구성
            details = []
            
            # 주소 정보
            address = person.get("address", {})
            if address:
                details.append(
                    f"주소: {address.get('city')} {address.get('street')} ({address.get('zipCode')})"
                )
            
            # 취미 정보
            hobbies = person.get("hobbies", [])
            if hobbies:
                details.append(f"취미: {', '.join(hobbies)}")
            
            # 연락처 정보
            phone_numbers = person.get("phoneNumbers", [])
            if phone_numbers:
                details.append(f"연락처: {', '.join(phone_numbers)}")
            
            # 상태 정보
            status = []
            if "isMarried" in person:
                status.append(f"결혼여부: {'기혼' if person['isMarried'] else '미혼'}")
            if "carOwnership" in person:
                status.append(f"차량소유: {'있음' if person['carOwnership'] else '없음'}")
            
            if status:
                details.append(", ".join(status))
            
            # 전체 텍스트 생성
            full_text = "\n".join(details)
            
            # 텍스트 분할
            chunks = text_splitter.split_text(full_text)
            
            # 각 청크에 기본 정보 추가
            for i, chunk in enumerate(chunks):
                chunk_data = {
                    **basic_info,
                    "chunk_index": i,
                    "total_chunks": len(chunks),
                    "content": chunk
                }
                chunked_data.append(chunk_data)
                
        except Exception as e:
            print(f"프로필 처리 중 에러 발생 (이름: {person.get('name')}): {str(e)}")
            continue
    
    return chunked_data

# 파일 처리 실행
def main():
    file_path = "data/people.json"
    
    # 데이터 로드 및 분할
    result = load_and_split_people_data(file_path)
    
    if result:
        print(f"\n총 {len(result)}개의 청크로 분할됨")
        
        # 첫 번째 청크 출력 (예시)
        if result:
            print("\n첫 번째 청크 예시:")
            print(json.dumps(result[0], ensure_ascii=False, indent=2))
            
        # 분할된 데이터 저장 (선택사항)
        output_path = "split_people_data.json"
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=2)
            print(f"\n분할된 데이터가 {output_path}에 저장되었습니다.")
        except Exception as e:
            print(f"데이터 저장 중 에러 발생: {str(e)}")

if __name__ == "__main__":
    main()

데이터 로드 완료: 20개의 프로필

총 20개의 청크로 분할됨

첫 번째 청크 예시:
{
  "name": "박시우",
  "age": 31,
  "chunk_index": 0,
  "total_chunks": 1,
  "content": "주소: 서울 312번지 (83795)\n취미: 요리, 음악 감상, 사진 촬영\n연락처: 483-4639-1933, 947-4179-7976\n결혼여부: 기혼, 차량소유: 있음"
}

분할된 데이터가 split_people_data.json에 저장되었습니다.
